In [1]:
import os
import pandas as pd

In [2]:
output_files = os.listdir("./evaluation_1k/")

In [3]:
results = pd.DataFrame(
    columns=[
        "Embedding Method",
        "Supporting documents retrieved",
        "Query Type",
        "Condition skyline (p@k)",
        "LLM",
        "Conditions accuracy",
        "Severity accuracy",
    ],
)

In [4]:
query_types = ["all", "cluster", "vague", "hypochondriac", "basic", "downplay"]

In [5]:
for output_file in output_files:
    # Get the model name and k value from the filename
    output_file_split = output_file.split("_")[0].split("-")
    model = "-".join(output_file_split[2:4]).replace("pt", ".")
    k = output_file_split[4][1:]
    print(model, k)

    # Read the jsonl file
    print(f"Reading './evaluation_1k/{output_file}'")
    data = pd.read_json(f"./evaluation_1k/{output_file}", lines=True)

    for query_type in query_types:
        if query_type == "all":
            sub_data = data
        else:
            # Get data for the specific query type
            sub_data = data[data["query_type"] == query_type]
        condition_match, severity_match, retriever_match = sub_data[
            ["conditions_match", "severity_match", "retriever_match"]
        ].sum(axis=0) / len(sub_data)

        i = len(results)
        results.loc[i, "Supporting documents retrieved"] = k
        results.loc[i, "Query Type"] = query_type
        results.loc[i, "Condition skyline (p@k)"] = retriever_match
        results.loc[i, "LLM"] = model
        results.loc[i, "Conditions accuracy"] = condition_match
        results.loc[i, "Severity accuracy"] = severity_match

In [6]:
results.sort_values(
    by=["Query Type", "Supporting documents retrieved", "LLM"], inplace=True
)
results["Embedding Method"] = "sentence-transformers/all-mpnet-base-v2"

In [8]:
results.to_csv(
    "./2b_granular.csv",
    index=False,
)

In [9]:
!uv pip install tabulate

md_text = """
## 2b_580a395.md split by query type

"""

for query_type in query_types:
    sub_results = results[results["Query Type"] == query_type]

    # add to markdown
    md_text += f"### {query_type}\n"
    md_text += sub_results.to_markdown(index=False, tablefmt="github", floatfmt=".2f")
    md_text += "\n\n"

with open("./2b_granular.md", "w") as f:
    f.write(md_text)

Using Python 3.12.9 environment at: /Users/rwood/projects/t0_proj/t0/experiments/t0/.venv
Audited 1 package in 8ms
